# 결측치 채우기

결과: 
k-세대타입(분양형태)              876125 -> 634715

k-난방방식                       876125 -> 635532

k-단지분류(아파트,주상복합등등)     877273 -> 635503

k-연면적                         876125 -> **32396**

k-전체세대수                      876125 -> **32396**

k-건설사(시공사)                  877637 -> 634666

k-시행사                         877834 -> 634711

k-홈페이지                       1013523 -> 662192

k-관리방식                       876125 -> 634455

k-복도유형                       876454 -> 635151

k-관리비부과면적                  876125 -> 634348

k-주거전용면적                    876170 -> 634393

경비비관리형태                    877561 -> 638950

세대전기계약방법                  885389 -> 639350

청소비관리형태                    877751 -> 638350

k-팩스번호                       879348 -> 634688

k-전화번호                       876850 -> 634348

주차대수                         876277 -> 637444

output:
1. train_restore.csv(upstage-ml-regression-ml_10/Datas/train_restore.csv)
2. test_restore.csv(upstage-ml-regression-ml_10/Datas/test_restore.csv)

In [8]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
import re

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [9]:
def normalize_apartment_name(apt_name: str) -> str:
    """
    아파트명을 정규화하는 함수
    
    Args:
        apt_name (str): 정규화할 아파트명
        
    Returns:
        str: 정규화된 아파트명
    """
    if pd.isna(apt_name):
        return ""
    
    # 문자열로 변환
    apt_name = str(apt_name)
    
    # 공백 제거 및 소문자 변환
    apt_name = apt_name.strip().lower()
    
    # 괄호와 그 안의 내용 제거 (예: (1단지), (2차) 등)
    apt_name = re.sub(r'\([^)]*\)', '', apt_name)
    
    # 특수문자 제거
    apt_name = re.sub(r'[^\w\s]', '', apt_name)
    
    # 연속된 공백을 하나로 통일
    apt_name = re.sub(r'\s+', ' ', apt_name)
    
    # 서울시 구 제거 (구가 붙은 경우와 안 붙은 경우 모두)
    gu_patterns = [
        r'강남(구)?',
        r'강동(구)?',
        r'강북(구)?',
        r'강서(구)?',
        r'관악(구)?',
        r'광진(구)?',
        r'구로(구)?',
        r'금천(구)?',
        r'노원(구)?',
        r'도봉(구)?',
        r'동대문(구)?',
        r'동작(구)?',
        r'마포(구)?',
        r'서대문(구)?',
        r'서초(구)?',
        r'성동(구)?',
        r'성북(구)?',
        r'송파(구)?',
        r'양천(구)?',
        r'영등포(구)?',
        r'용산(구)?',
        r'은평(구)?',
        r'종로(구)?',
        r'중(구)?',
        r'중랑(구)?'
    ]
    
    # 서울시 모든 법정동 제거 (동이 붙은 경우와 안 붙은 경우 모두)
    dong_patterns = [
        # 강남구
        r'개포(동)?', r'논현(동)?', r'대치(동)?', r'도곡(동)?', r'삼성(동)?', r'세곡(동)?', r'신사(동)?', r'압구정(동)?', r'역삼(동)?', r'일원(동)?', r'청담(동)?',
        # 강동구
        r'강일(동)?', r'고덕(동)?', r'길(동)?', r'둔촌(동)?', r'명일(동)?', r'상일(동)?', r'성내(동)?', r'암사(동)?', r'천호(동)?',
        # 강북구
        r'미아(동)?', r'번(동)?', r'수유(동)?', r'우이(동)?',
        # 강서구
        r'가양(동)?', r'개화(동)?', r'공항(동)?', r'과해(동)?', r'내발산(동)?', r'등촌(동)?', r'마곡(동)?', r'방화(동)?', r'오곡(동)?', r'오쇠(동)?', r'외발산(동)?', r'화곡(동)?',
        # 관악구
        r'남현(동)?', r'봉천(동)?', r'신림(동)?',
        # 광진구
        r'광장(동)?', r'구의(동)?', r'군자(동)?', r'능(동)?', r'자양(동)?', r'중곡(동)?', r'화양(동)?',
        # 구로구
        r'가리봉(동)?', r'개봉(동)?', r'고척(동)?', r'구로(동)?', r'신도림(동)?', r'오류(동)?', r'천왕(동)?', r'항(동)?',
        # 금천구
        r'가산(동)?', r'독산(동)?', r'시흥(동)?',
        # 노원구
        r'공릉(동)?', r'상계(동)?', r'월계(동)?', r'중계(동)?', r'하계(동)?',
        # 도봉구
        r'도봉(동)?', r'방학(동)?', r'쌍문(동)?', r'창(동)?',
        # 동대문구
        r'답십리(동)?', r'신설(동)?', r'용두(동)?', r'이문(동)?', r'장안(동)?', r'전농(동)?', r'제기(동)?', r'청량리(동)?', r'회기(동)?', r'휘경(동)?',
        # 동작구
        r'노량진(동)?', r'대방(동)?', r'동작(동)?', r'본(동)?', r'사당(동)?', r'상도(동)?', r'신대방(동)?', r'흑석(동)?',
        # 마포구
        r'공덕(동)?', r'구수(동)?', r'노고산(동)?', r'당인(동)?', r'대흥(동)?', r'도화(동)?', r'동교(동)?', r'마포(동)?', r'망원(동)?', r'상암(동)?', r'서교(동)?', r'성산(동)?', r'신수(동)?', r'연남(동)?', r'염리(동)?', r'용강(동)?', r'중(동)?', r'창전(동)?', r'토정(동)?', r'하중(동)?', r'합정(동)?', r'현석(동)?',
        # 서대문구
        r'남가좌(동)?', r'냉천(동)?', r'대신(동)?', r'대현(동)?', r'미근(동)?', r'봉원(동)?', r'북가좌(동)?', r'북아현(동)?', r'신촌(동)?', r'연희(동)?', r'영천(동)?', r'옥천(동)?', r'창천(동)?', r'천연(동)?', r'충정로(동)?', r'합(동)?', r'현저(동)?', r'홍은(동)?', r'홍제(동)?',
        # 서초구
        r'반포(동)?', r'방배(동)?', r'서초(동)?', r'양재(동)?', r'우면(동)?', r'잠원(동)?',
        # 성동구
        r'금호(동)?', r'도선(동)?', r'마장(동)?', r'사근(동)?', r'상왕십리(동)?', r'성수(동)?', r'송정(동)?', r'옥수(동)?', r'왕십리(동)?', r'용답(동)?', r'응봉(동)?', r'하왕십리(동)?', r'행당(동)?', r'홍익(동)?',
        # 성북구
        r'길음(동)?', r'돈암(동)?', r'동선(동)?', r'동소문(동)?', r'보문(동)?', r'삼선(동)?', r'상월곡(동)?', r'석관(동)?', r'성북(동)?', r'안암(동)?', r'장위(동)?', r'정릉(동)?', r'종암(동)?', r'하월곡(동)?',
        # 송파구
        r'가락(동)?', r'문정(동)?', r'방이(동)?', r'송파(동)?', r'신천(동)?', r'잠실(동)?', r'장지(동)?', r'풍납(동)?',
        # 양천구
        r'목(동)?', r'신월(동)?', r'신정(동)?',
        # 영등포구
        r'당산(동)?', r'대림(동)?', r'도림(동)?', r'문래(동)?', r'신길(동)?', r'양평(동)?', r'여의도(동)?', r'영등포(동)?', r'용산(동)?',
        # 용산구
        r'갈월(동)?', r'남영(동)?', r'도원(동)?', r'문배(동)?', r'보광(동)?', r'산천(동)?', r'서계(동)?', r'용문(동)?', r'원효로(동)?', r'이촌(동)?', r'이태원(동)?', r'장충(동)?', r'청파(동)?', r'한강로(동)?', r'한남(동)?', r'효창(동)?', r'후암(동)?',
        # 은평구
        r'갈현(동)?', r'구산(동)?', r'녹번(동)?', r'대조(동)?', r'불광(동)?', r'수색(동)?', r'신사(동)?', r'역촌(동)?', r'응암(동)?', r'증산(동)?', r'진관(동)?',
        # 종로구
        r'가회(동)?', r'견지(동)?', r'경운(동)?', r'계(동)?', r'공평(동)?', r'관수(동)?', r'관철(동)?', r'관훈(동)?', r'교남(동)?', r'교북(동)?', r'교서(동)?', r'교촌(동)?', r'내수(동)?', r'내자(동)?', r'누상(동)?', r'누하(동)?', r'당주(동)?', r'도렴(동)?', r'돈의(동)?', r'동숭(동)?', r'명륜(동)?', r'묘(동)?', r'봉익(동)?', r'부암(동)?', r'사간(동)?', r'사직(동)?', r'삼청(동)?', r'서린(동)?', r'세종로', r'소격(동)?', r'송월(동)?', r'송현(동)?', r'수송(동)?', r'숭인(동)?', r'신교(동)?', r'신문로', r'신영(동)?', r'안국(동)?', r'연건(동)?', r'연지(동)?', r'예지(동)?', r'옥인(동)?', r'와룡(동)?', r'운니(동)?', r'원남(동)?', r'원서(동)?', r'이화(동)?', r'익선(동)?', r'인사(동)?', r'인의(동)?', r'장사(동)?', r'재(동)?', r'적선(동)?', r'종로(동)?', r'중학(동)?', r'창성(동)?', r'창신(동)?', r'청진(동)?', r'체부(동)?', r'충신(동)?', r'통의(동)?', r'통인(동)?', r'팔판(동)?', r'평(동)?', r'평창(동)?', r'필운(동)?', r'행촌(동)?', r'혜화(동)?', r'화(동)?', r'효자(동)?', r'효제(동)?', r'훈정(동)?',
        # 중구
        r'광희(동)?', r'남대문로', r'남산(동)?', r'남창(동)?', r'남학(동)?', r'다(동)?', r'만리(동)?', r'명(동)?', r'무교(동)?', r'무학(동)?', r'묵정(동)?', r'방산(동)?', r'봉래(동)?', r'북창(동)?', r'산림(동)?', r'삼각(동)?', r'서소문(동)?', r'소공(동)?', r'수표(동)?', r'수하(동)?', r'순화(동)?', r'신당(동)?', r'쌍림(동)?', r'예관(동)?', r'예장(동)?', r'오장(동)?', r'을지로', r'의주로', r'인현(동)?', r'입정(동)?', r'장교(동)?', r'장충(동)?', r'저(동)?', r'정(동)?', r'주교(동)?', r'주자(동)?', r'중림(동)?', r'초(동)?', r'충무로', r'충정로', r'태평로', r'필(동)?', r'황학(동)?', r'회현(동)?', r'흥인(동)?',
        # 중랑구
        r'망우(동)?', r'면목(동)?', r'묵(동)?', r'상봉(동)?', r'신내(동)?', r'중화(동)?'
    ]
    
    # 구 제거
    for pattern in gu_patterns:
        apt_name = re.sub(pattern, '', apt_name, flags=re.IGNORECASE)
    
    # 동 제거
    for pattern in dong_patterns:
        apt_name = re.sub(pattern, '', apt_name, flags=re.IGNORECASE)
    
    # 아파트 관련 단어 통일
    replacements = {
        '아파트': '아파트',
        'apt': '아파트',
        'a.p.t': '아파트',
        'a.p.t.': '아파트',
        'a p t': '아파트',
        'a p t.': '아파트',
        'a.p.t': '아파트',
        'a.p.t.': '아파트',
        'a p t': '아파트',
        'a p t.': '아파트',
        '아파트먼트': '아파트',
        'apartment': '아파트',
        'apart': '아파트'
    }
    
    for old, new in replacements.items():
        apt_name = apt_name.replace(old.lower(), new)
    
    # 차수 이동 (예: 3차신성미소지움 -> 신성미소지움3차)
    phase_pattern = re.compile(r'(\d+)차')
    match = phase_pattern.search(apt_name)
    if match:
        phase = match.group(0)  # 예: "3차"
        apt_name = apt_name.replace(phase, '')  # 차수 제거
        apt_name = apt_name + phase  # 차수를 뒤에 추가
    
    # 모든 공백 제거
    apt_name = apt_name.replace(' ', '')
    
    # 최종 공백 제거
    apt_name = apt_name.strip()
    
    return apt_name

In [10]:
def normalize_apartment_names_in_dataframe(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    DataFrame의 특정 컬럼에 있는 아파트명들을 정규화하는 함수
    
    Args:
        df (pd.DataFrame): 정규화할 DataFrame
        column_name (str): 아파트명이 있는 컬럼명
        
    Returns:
        pd.DataFrame: 정규화된 아파트명이 적용된 DataFrame
    """
    df = df.copy()
    df[column_name+"_normalized"] = df[column_name].apply(normalize_apartment_name)
    return df 

In [11]:
# 본인의 데이터셋 경로를 이용

# house_train = pd.read_csv("/root/datas/data/train.csv")
# house_test = pd.read_csv("/root/datas/data/test.csv")
# house_train['is_test'] = 0
# house_test['is_test'] = 1

# df = pd.concat([house_train, house_test])
df = pd.read_csv("./../../../Datas/updated_df(knn).csv")
# df = df[["시군구", "아파트명"]]


df_original = df.copy()

In [33]:
df1 = pd.read_csv("/root/datas/upstage-ml-regression-ml_10/Datas/updated_df2(knn).csv")
display(df1)

,Unnamed: 0,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,...,아파트군,한강변,구,동,계약연도,계약월,구_평균가격순위,구_상승률순위,신축여부,건물연식
0,0,서울특별시 강남구 개포동,658-1,658.0,1,개포6차우성,79.97,201712,8,3,...,2,1,강남구,개포동,2017,12,1.0,6.0,0,30
1,1,서울특별시 강남구 개포동,658-1,658.0,1,개포6차우성,79.97,201712,22,4,...,2,1,강남구,개포동,2017,12,1.0,6.0,0,30
2,2,서울특별시 강남구 개포동,658-1,658.0,1,개포6차우성,54.98,201712,28,5,...,2,1,강남구,개포동,2017,12,1.0,6.0,0,30
3,3,서울특별시 강남구 개포동,658-1,658.0,1,개포6차우성,79.97,201801,3,4,...,2,1,강남구,개포동,2018,1,1.0,6.0,0,31
4,4,서울특별시 강남구 개포동,658-1,658.0,1,개포6차우성,79.97,201801,8,2,...,2,1,강남구,개포동,2018,1,1.0,6.0,0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,1128089,서울특별시 중랑구 신내동,816,816.0,0,신내우디안1단지,84.65,202307,19,13,...,1,0,중랑구,신내동,2023,7,22.0,18.0,1,9
1128090,1128090,서울특별시 중랑구 신내동,816,816.0,0,신내우디안1단지,84.62,202307,25,12,...,1,0,중랑구,신내동,2023,7,22.0,18.0,1,9
1128091,1128091,서울특별시 중랑구 신내동,816,816.0,0,신내우디안1단지,101.65,202308,27,12,...,1,0,중랑구,신내동,2023,8,22.0,18.0,1,9
1128092,1128092,서울특별시 중랑구 신내동,816,816.0,0,신내우디안1단지,84.94,202309,2,18,...,1,0,중랑구,신내동,2023,9,22.0,18.0,1,9


## 원본 데이터셋에 결측치를 처리하기위한 데이터셋 merge

아파트명과 도로명을 동시에 사용하여 key값으로 하여 merge를 할 것이다. 이때 아파트명은 정규화를 한다.
그런데 아파트명 도로명을 동시에 사용하여 key값으로 해도 중복되는 데이터가 있는데 이것은 분양, 임대로 인한 차이로 인해 생기는 것이다.

In [12]:
#아파트 기본정보 데이터셋 불러오기

basic_df = pd.read_csv("./../../../Datas/basic_df.csv")
basic = basic_df.copy()
basic['도로명'] = basic['doroJuso'].str.split(' ').str[2:].str.join(' ').str.strip()
cols = list(basic.columns)
cols[1] = '아파트명'
basic.columns = cols

#제거할 아파트(분양, 임대 차이로 인한 중복 처리)
drop_aptName = ['인왕산현대', '성북동아에코빌', 'DMC래미안클라시스']
drop_aptCode = basic[basic['아파트명'].isin(drop_aptName)]['kaptCode'].unique()


#아파트 상세정보 데이터셋 불러오기
detail_df = pd.read_csv("./../../../Datas/detail_df.csv")
detail = detail_df.copy()

#아파트 기본정보와 상세정보 merge(on kaptCode)
apart_info = basic.merge(detail, on='kaptCode', how='left')

#불필요한 아파트 제거
apart_info = apart_info[~apart_info['kaptCode'].isin(drop_aptCode)].reset_index(drop=True)

#아파트명 정규화
aparts_df = normalize_apartment_names_in_dataframe(apart_info, '아파트명')
display(aparts_df)



,kaptCode,아파트명,kaptAddr,codeSaleNm,codeHeatNm,kaptTarea,kaptDongCnt,kaptdaCnt,kaptBcompany,kaptAcompany,...,welfareFacility,kaptdWtimebus,subwayLine,subwayStation,kaptdWtimesub,convenientFacility,educationFacility,groundElChargerCnt,undergroundElChargerCnt,아파트명_normalized
0,A10021295,경희궁의아침4단지,서울특별시 종로구 내수동 73- 경희궁의아침4단지,분양,개별난방,27557.00,1,120.0,쌍용건설,군인공제회,...,NaN,5~10분이내,NaN,NaN,NaN,NaN,NaN,0,4,경희궁의아침4단지
1,A10021652,경희궁파크팰리스,서울특별시 종로구 내수동 95- 경희궁파크팰리스,분양,개별난방,41113.00,1,142.0,삼성물산,확인불가,...,NaN,5~10분이내,NaN,NaN,NaN,NaN,NaN,0,3,경희궁파크팰리스
2,A11007001,경희궁의아침3단지,서울특별시 종로구 내수동 72 경희궁의아침3단지,분양,개별난방,38391.60,1,150.0,쌍용건설,군인공제조합,...,"관리사무소, 주민공동시설",5~10분이내,"3호선, 5호선",경복궁역,5~10분이내,관공서(서울지방경찰청) 병원(곽치과의원) 공원(경희궁공원),초등학교(매동초교) 중학교(배화여중) 고등학교(배화여고) 대학교(배화여대),0,2,경희궁의아침3단지
3,A11087101,경희궁의아침2단지,서울특별시 종로구 내수동 71 경희궁의아침2단지,분양,개별난방,47470.58,1,90.0,쌍용건설,쌍용건설,...,NaN,5~10분이내,3호선,경복궁역,5~10분이내,관공서(서울지방경찰청) 병원(곽치과의원) 공원(경희궁공원),초등학교(매동초교) 중학교(배화여중) 고등학교(배화여고) 대학교(배화여대),0,0,경희궁의아침2단지
4,A11005401,광화문스페이스본 아파트,서울특별시 종로구 신문로2가 1-434 광화문스페이스본 아파트,분양,NaN,163862.10,6,744.0,풍림산업(주),사직1구역 도심 재개발조합,...,"관리사무소, 보육시설, 문고, 주민공동시설, 어린이놀이터, 휴게시설, 커뮤니티공간,...",5분이내,"3호선, 5호선",NaN,5~10분이내,"관공서(서울특별시청, 종로구청) 병원(서대문구보건소, 강북삼성병원, 적십자병원, 천...","초등학교(매동초,대신초) 중학교(배화여자중,청운중) 고등학교(배화여자고,덕성여고) ...",0,135,광문스페이스아파트
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,A13410008,강일리버파크3단지,서울특별시 강동구 강일동 674 강일리버파크3단지,혼합,지역난방,112365.71,13,987.0,삼환기업,sh공사,...,"관리사무소, 노인정, 보육시설, 주민공동시설, 어린이놀이터, 자전거보관소, 기타",5분이내,5호선,NaN,15~20분이내,관공서() 병원() 공원(),초등학교() 중학교() 고등학교(),16,6,리버파크3단지
3272,A13410009,강일리버파크5단지,서울특별시 강동구 강일동 670 강일리버파크5단지,혼합,지역난방,84262.00,9,722.0,KR산업,SH공사,...,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터",5분이내,5호선,NaN,10~15분이내,NaN,NaN,17,0,리버파크5단지
3273,A13410010,강일리버파크7단지,서울특별시 강동구 강일동 686 강일리버파크7단지,혼합,지역난방,80752.58,11,731.0,고속도로관리공단,SH공사,...,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 자전거보관소",5분이내,5호선,NaN,20분초과,NaN,초등학교() 고등학교(),10,7,리버파크7단지
3274,A13410011,고덕리엔파크2단지,서울특별시 강동구 강일동 717 고덕리엔파크2단지,혼합,지역난방,87367.62,11,636.0,신동아,SH공사,...,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 휴게시설, 자전...",5분이내,5호선,강일역,5~10분이내,병원(경희대 동서신의학병원) 대형상가(이마트),"초등학교(강일, 고덕, 상일초) 중학교(강일, 고덕, 배재중) 고등학교(배재,강일고...",0,3,리엔파크2단지


In [13]:
#아파트명 별 도로명 unique 
df['도로명'] = df['도로명'].str.strip()
df_unique = df.groupby(['아파트명', '도로명'])['k-세대타입(분양형태)'].unique().reset_index(name='분양형태')
df_unique = normalize_apartment_names_in_dataframe(df_unique, '아파트명')
display(df_unique)


,아파트명,도로명,분양형태,아파트명_normalized
0,153-25,북아현로1가길 23,[nan],15325
1,201호 미공시,창신길 20,[nan],201호미공시
2,2차한양,가락로 192,[분양],한양2차
3,3RU-City,신촌로 1,[nan],3rucity
4,3차신성미소지움,오목로 39-22,[nan],신성미소지움3차
...,...,...,...,...
9392,힐튼빌리지2차,후암로16나길 23,[nan],힐튼빌리지2차
9393,힐하우스,동광로46길 7-15,[nan],힐하우스
9394,힐하우스,동산로12길 5-8,[nan],힐하우스
9395,힐하우스,방배로34길 86,[nan],힐하우스


In [14]:
# 도로명 주소를 기준으로 df_unique 와 basic 조인
# 첫 번째 merge: 도로명 기준
merge_df = df_unique.merge(aparts_df, on=['도로명', '아파트명_normalized'], how='left')
display(merge_df)

,아파트명_x,도로명,분양형태,아파트명_normalized,kaptCode,아파트명_y,kaptAddr,codeSaleNm,codeHeatNm,kaptTarea,...,kaptdCccnt,welfareFacility,kaptdWtimebus,subwayLine,subwayStation,kaptdWtimesub,convenientFacility,educationFacility,groundElChargerCnt,undergroundElChargerCnt
0,153-25,북아현로1가길 23,[nan],15325,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201호 미공시,창신길 20,[nan],201호미공시,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2차한양,가락로 192,[분양],한양2차,A13885304,송파한양2차,서울특별시 송파구 송파동 151 송파한양2차,분양,지역난방,98677.53,...,44.0,"관리사무소, 노인정, 어린이놀이터, 자전거보관소",5~10분이내,"8호선, 9호선",송파,15~20분이내,"관공서(송파2동사무소, 송파경찰서) 병원(국립경찰병원, 서울병원) 백화점(롯데 잠실...","초등학교(중대) 중학교(가락) 고등학교(가락고, 잠실여고)",3.0,0.0
3,3RU-City,신촌로 1,[nan],3rucity,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3차신성미소지움,오목로 39-22,[nan],신성미소지움3차,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9392,힐튼빌리지2차,후암로16나길 23,[nan],힐튼빌리지2차,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9393,힐하우스,동광로46길 7-15,[nan],힐하우스,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9394,힐하우스,동산로12길 5-8,[nan],힐하우스,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9395,힐하우스,방배로34길 86,[nan],힐하우스,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# 병합된 데이터셋의 이름 처리 및 drop
# merge_df: 원본 데이터셋의 아파트명과 도로명에 따른 basic 데이터셋의 정보

merge_df_cols = list(merge_df.columns)
merge_df_cols[0] = '아파트명'
merge_df.columns = merge_df_cols
merge_df.drop(columns=['아파트명_normalized', '분양형태', '아파트명_y'], inplace=True)
display(merge_df)

,아파트명,도로명,kaptCode,kaptAddr,codeSaleNm,codeHeatNm,kaptTarea,kaptDongCnt,kaptdaCnt,kaptBcompany,...,kaptdCccnt,welfareFacility,kaptdWtimebus,subwayLine,subwayStation,kaptdWtimesub,convenientFacility,educationFacility,groundElChargerCnt,undergroundElChargerCnt
0,153-25,북아현로1가길 23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201호 미공시,창신길 20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2차한양,가락로 192,A13885304,서울특별시 송파구 송파동 151 송파한양2차,분양,지역난방,98677.53,10.0,744.0,(주)한양,...,44.0,"관리사무소, 노인정, 어린이놀이터, 자전거보관소",5~10분이내,"8호선, 9호선",송파,15~20분이내,"관공서(송파2동사무소, 송파경찰서) 병원(국립경찰병원, 서울병원) 백화점(롯데 잠실...","초등학교(중대) 중학교(가락) 고등학교(가락고, 잠실여고)",3.0,0.0
3,3RU-City,신촌로 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3차신성미소지움,오목로 39-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9392,힐튼빌리지2차,후암로16나길 23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9393,힐하우스,동광로46길 7-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9394,힐하우스,동산로12길 5-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9395,힐하우스,방배로34길 86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# merge_df column 이름 변환
#'kaptCode', 'kaptAddr', 'codeSaleNm', 'codeHeatNm',
#      'kaptTarea', 'kaptDongCnt', 'kaptdaCnt', 'kaptBcompany', 'kaptAcompany',
#       'kaptTel', 'kaptUrl', 'codeAptNm', 'doroJuso', 'codeMgrNm',
#       'codeHallNm', 'kaptUsedate', 'kaptFax', 'hoCnt', 'kaptMarea',
#       'kaptMparea60', 'kaptMparea85', 'kaptMparea135', 'kaptMparea136',
#       'privArea', 'bjdCode', 'kaptTopFloor', 'ktownFlrNo', 'kaptBaseFloor',
#       'kaptdEcntp', 'zipcode'
#법정동주소, 분양형태, 난방방식, 건축물대장상 연면적, 동수, 세대수, 시공사, 시행사, 관리사무소연락처, 관리사무소팩스, 홈페이지주소, 단지분류, 도로명주소, 호수, 관리방식, 복도유형, 사용승인일, 관리비부과면적, 전용면적별 세대현황, 단지 전용면적합, 법정동코드
changes = {
    'codeSaleNm': 'k-세대타입(분양형태)',
    'codeHeatNm': 'k-난방방식',
    'codeAptNm': 'k-단지분류(아파트,주상복합등등)',
    'kaptTarea': 'k-연면적',
    'kaptDongCnt': 'k-전체동수',
    'kaptdaCnt': 'k-전체세대수',
    'kaptBcompany': 'k-건설사(시공사)',
    'kaptAcompany': 'k-시행사',
    'kaptUrl': 'k-홈페이지',
    'codeMgrNm': 'k-관리방식',
    'codeHallNm': 'k-복도유형',
    'kaptMarea': 'k-관리비부과면적',
    'privArea': 'k-주거전용면적',
    'codeSec': '경비비관리형태',
    'codeEcon': '세대전기계약방법',
    'codeClean': '청소비관리형태',
    'kaptFax': 'k-팩스번호',
    'kaptTel': 'k-전화번호',
    'kaptdPcnt': '주차대수(지상)',
    'kaptdPcntu': '주차대수(지하)',

    
}

merge_df.rename(columns=changes, inplace=True)

# 원본 데이터셋에 병합할 merge_df의 컬럼 선택
remain_cols = list(changes.values()) + ['아파트명', '도로명']
# remain_cols = np.array(['아파트명', '도로명', 'k-전체세대수','k-연면적','k-단지분류(아파트,주상복합등등)','k-세대타입(분양형태)','k-관리방식','k-복도유형','k-홈페이지','k-난방방식','k-건설사(시공사)','k-시행사','k-관리비부과면적', 'k-주거전용면적'])
merge_df = merge_df[remain_cols]
display(merge_df)

,k-세대타입(분양형태),k-난방방식,"k-단지분류(아파트,주상복합등등)",k-연면적,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-홈페이지,k-관리방식,...,k-주거전용면적,경비비관리형태,세대전기계약방법,청소비관리형태,k-팩스번호,k-전화번호,주차대수(지상),주차대수(지하),아파트명,도로명
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153-25,북아현로1가길 23
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201호 미공시,창신길 20
2,분양,지역난방,아파트,98677.53,10.0,744.0,(주)한양,(주)한양,,위탁관리,...,80631.31,위탁관리,단일계약,위탁관리,024208288,024237427,862.0,0.0,2차한양,가락로 192
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3RU-City,신촌로 1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3차신성미소지움,오목로 39-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,힐튼빌리지2차,후암로16나길 23
9393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,힐하우스,동광로46길 7-15
9394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,힐하우스,동산로12길 5-8
9395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,힐하우스,방배로34길 86


In [17]:
# 주차대수(지상)과 주차대수(지하)를 합쳐서 주차대수 column 생성
merge_df['주차대수'] = merge_df['주차대수(지상)'] + merge_df['주차대수(지하)']
print(merge_df['주차대수'].isnull().sum())
print(merge_df['주차대수(지상)'].isnull().sum())
print(merge_df['주차대수(지하)'].isnull().sum())
merge_df.drop(columns=['주차대수(지상)', '주차대수(지하)'], inplace=True)
display(merge_df)


8081
8080
8079


,k-세대타입(분양형태),k-난방방식,"k-단지분류(아파트,주상복합등등)",k-연면적,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-홈페이지,k-관리방식,...,k-관리비부과면적,k-주거전용면적,경비비관리형태,세대전기계약방법,청소비관리형태,k-팩스번호,k-전화번호,아파트명,도로명,주차대수
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153-25,북아현로1가길 23,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201호 미공시,창신길 20,NaN
2,분양,지역난방,아파트,98677.53,10.0,744.0,(주)한양,(주)한양,,위탁관리,...,80604.96,80631.31,위탁관리,단일계약,위탁관리,024208288,024237427,2차한양,가락로 192,862.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3RU-City,신촌로 1,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3차신성미소지움,오목로 39-22,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,힐튼빌리지2차,후암로16나길 23,NaN
9393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,힐하우스,동광로46길 7-15,NaN
9394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,힐하우스,동산로12길 5-8,NaN
9395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,힐하우스,방배로34길 86,NaN


In [18]:
print(len(df_original))

# 원본 데이터셋과 병합
df = df_original.merge(merge_df, on = ['아파트명', '도로명'], how = 'left', suffixes=('', '_y'))
print(len(df))


1128094
1128094


## 결측치 처리

In [19]:
# 만약 결측치가 있다면 _y가 붙은 컬럼으로 채워넣기

restore_cols = list(changes.values())[:-2] + ['주차대수']
# ho_idx = restore_cols.index('k-전체호수')
# print(ho_idx)
# restore_cols.pop(ho_idx)
print(restore_cols)
display(df.isnull().sum())
# 결측치 채우기
for col in restore_cols:
    df[col] = df[col].fillna(df[col + '_y'])
# 결측치 확인
display(df.isnull().sum())


['k-세대타입(분양형태)', 'k-난방방식', 'k-단지분류(아파트,주상복합등등)', 'k-연면적', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-홈페이지', 'k-관리방식', 'k-복도유형', 'k-관리비부과면적', 'k-주거전용면적', '경비비관리형태', '세대전기계약방법', '청소비관리형태', 'k-팩스번호', 'k-전화번호', '주차대수']


시군구                0
번지               227
본번                75
부번                 0
아파트명               0
               ...  
세대전기계약방법_y    698280
청소비관리형태_y     698335
k-팩스번호_y      695002
k-전화번호_y      695002
주차대수_y        698370
Length: 85, dtype: int64

시군구                0
번지               227
본번                75
부번                 0
아파트명               0
               ...  
세대전기계약방법_y    698280
청소비관리형태_y     698335
k-팩스번호_y      695002
k-전화번호_y      695002
주차대수_y        698370
Length: 85, dtype: int64

In [20]:
display(df[restore_cols].isnull().sum())

k-세대타입(분양형태)          634709
k-난방방식                635526
k-단지분류(아파트,주상복합등등)    635497
k-연면적                  32394
k-전체동수                634569
k-전체세대수                32394
k-건설사(시공사)            634660
k-시행사                 634857
k-홈페이지                662188
k-관리방식                634449
k-복도유형                635145
k-관리비부과면적             634342
k-주거전용면적              634387
경비비관리형태               638948
세대전기계약방법              639344
청소비관리형태               638344
k-팩스번호                634684
k-전화번호                634348
주차대수                  637438
dtype: int64

In [21]:
# 뒤에 _y가 붙은 컬럼 삭제
y_columns = [col for col in df.columns if col.endswith('_y')]
df = df.drop(columns=y_columns)
display(df)

,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,아파트군,한강변,구,동,계약연도,계약월,구_평균가격순위,구_상승률순위,신축여부,건물연식
0,서울특별시 강남구 개포동,658-1,658.0,1,개포6차우성,79.97,201712,8,3,1987,...,2,1,강남구,개포동,2017,12,1.0,6.0,0,30
1,서울특별시 강남구 개포동,658-1,658.0,1,개포6차우성,79.97,201712,22,4,1987,...,2,1,강남구,개포동,2017,12,1.0,6.0,0,30
2,서울특별시 강남구 개포동,658-1,658.0,1,개포6차우성,54.98,201712,28,5,1987,...,2,1,강남구,개포동,2017,12,1.0,6.0,0,30
3,서울특별시 강남구 개포동,658-1,658.0,1,개포6차우성,79.97,201801,3,4,1987,...,2,1,강남구,개포동,2018,1,1.0,6.0,0,31
4,서울특별시 강남구 개포동,658-1,658.0,1,개포6차우성,79.97,201801,8,2,1987,...,2,1,강남구,개포동,2018,1,1.0,6.0,0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,816.0,0,신내우디안1단지,84.65,202307,19,13,2014,...,1,0,중랑구,신내동,2023,7,22.0,18.0,1,9
1128090,서울특별시 중랑구 신내동,816,816.0,0,신내우디안1단지,84.62,202307,25,12,2014,...,1,0,중랑구,신내동,2023,7,22.0,18.0,1,9
1128091,서울특별시 중랑구 신내동,816,816.0,0,신내우디안1단지,101.65,202308,27,12,2014,...,1,0,중랑구,신내동,2023,8,22.0,18.0,1,9
1128092,서울특별시 중랑구 신내동,816,816.0,0,신내우디안1단지,84.94,202309,2,18,2014,...,1,0,중랑구,신내동,2023,9,22.0,18.0,1,9


## 건축물대장을 이용한 연면적, 전체세대수 결측치 채우기

In [22]:
building_register = pd.read_csv("./../../../Datas/서울시 건축물대장 표제부.csv", 
                               encoding='cp949',
                               encoding_errors='replace')
display(building_register)

,대지위치,시군구코드명,법정동코드명,대지구분코드명,주지번,부지번,특수지명,블록번호,로트번호,새주소도로코드명,...,사용승인일자,에너지효율등급값,에너지절감률,EPI점수,친환경건축물등급값,친환경건축물인증점수,지능형건축물등급값,지능형건축물인증점수,내진설계적용여부,내진능력내용
0,서울특별시 강서구 방화동 246-66,서울특별시 강서구,방화동,대지,246,66,NaN,NaN,NaN,방화대로,...,1989-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,서울특별시 마포구 아현동 751-11,서울특별시 마포구,아현동,대지,751,11,NaN,NaN,NaN,환일길,...,1984-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,서울특별시 양천구 신월동 987-1,서울특별시 양천구,신월동,대지,987,1,NaN,NaN,NaN,신월로,...,1988-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,서울특별시 양천구 신정동 87-39,서울특별시 양천구,신정동,대지,87,39,NaN,NaN,NaN,신목로,...,1990-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,서울특별시 강서구 화곡동 102-29,서울특별시 강서구,화곡동,대지,102,29,NaN,NaN,NaN,까치산로4길,...,1990-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590960,서울특별시 도봉구 도봉동 594-13,서울특별시 도봉구,도봉동,대지,594,13,NaN,NaN,NaN,도봉로169길,...,1992-01-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
590961,서울특별시 노원구 공릉동 323-35,서울특별시 노원구,공릉동,대지,323,35,NaN,NaN,NaN,동일로,...,1996-07-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
590962,서울특별시 도봉구 창동 525-9,서울특별시 도봉구,창동,대지,525,9,NaN,NaN,NaN,우이천로,...,1974-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
590963,서울특별시 강남구 신사동 629-7,서울특별시 강남구,신사동,대지,629,7,NaN,NaN,NaN,언주로151길,...,2002-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [23]:
# 사용할만한 변수 추출
use_cols = np.array(['대지위치', '연면적', '구조코드명','기타용도내용','세대수','지상층수','부속건축물수', '총동연면적'])
building_register = building_register[use_cols]
use_cols[0] = '지역'
building_register.columns = use_cols
display(building_register)

,지역,연면적,구조코드명,기타용도내용,세대수,지상층수,부속건축물수,총동연면적
0,서울특별시 강서구 방화동 246-66,396.72,철근콘크리트구조,주택 및 창고,NaN,3.0,NaN,396.72
1,서울특별시 마포구 아현동 751-11,470.16,철근콘크리트구조,연립주택,6.0,3.0,NaN,470.16
2,서울특별시 양천구 신월동 987-1,7266.90,철근콘크리트구조,아파트,96.0,12.0,0.0,7266.90
3,서울특별시 양천구 신정동 87-39,445.12,철근콘크리트구조,근린생활시설,0.0,2.0,0.0,445.12
4,서울특별시 강서구 화곡동 102-29,247.87,벽돌구조,단독주택,0.0,2.0,0.0,247.87
...,...,...,...,...,...,...,...,...
590960,서울특별시 도봉구 도봉동 594-13,186.25,벽돌구조,단독주택(다가구주택),0.0,2.0,0.0,186.25
590961,서울특별시 노원구 공릉동 323-35,5009.25,철근콘크리트구조,"근린생활시설, 업무시설",0.0,6.0,2.0,5009.25
590962,서울특별시 도봉구 창동 525-9,60.30,벽돌구조,연립주택,2.0,1.0,0.0,60.30
590963,서울특별시 강남구 신사동 629-7,738.55,철근콘크리트구조,주거시설 근린생활시설,0.0,6.0,0.0,738.55


In [24]:
# 지역으로 그룹핑
restore = building_register.groupby('지역')[['연면적', '세대수']].sum().reset_index()
display(restore)

,지역,연면적,세대수
0,경기도 군포시 당동 921,783.76,0.0
1,서울시관악구 봉천동 950-26,77.58,0.0
2,서울시노원구 상계동 2구역22블럭44롯트,348.50,0.0
3,서울시영등포구 신길동 114-39,140.31,0.0
4,서울시은평구 수색동 370-4,54.25,0.0
...,...,...,...
517948,서울특별시 중랑구 중화동 99-2,248.94,0.0
517949,서울특별시 중랑구 중화동 99-3,298.02,6.0
517950,서울특별시 중랑구 중화동 99-4,452.27,10.0
517951,서울특별시 중랑구 중화동 99-5,217.94,0.0


In [25]:
# 지역 변수 생성
df['지역'] = df['시군구'] + ' ' + df['본번'].fillna(0).astype(int).astype(str)
df['부번'] = df['부번'].fillna(0).astype(int).astype(str)
df['지역'] = df.apply(lambda x: x['지역'] + '-' + x['부번'] if x['부번'] != '0' else x['지역'], axis=1)

In [26]:
# 원본 데이터와 restore 데이터 병합
df = df.merge(restore, on='지역', how='left')
# NaN과 0 값을 모두 대체
df['k-연면적'] = df['k-연면적'].replace(0, np.nan).fillna(df['연면적'])
df['k-전체세대수'] = df['k-전체세대수'].replace(0, np.nan).fillna(df['세대수'])

df.drop(columns=['연면적', '세대수'], inplace=True)
print(df['k-연면적'].isnull().sum())
print(df['k-전체세대수'].isnull().sum())

32394
32394


In [27]:
df['등기신청일자'] = df['등기신청일자'].replace('-', np.nan)
df['거래유형'] = df['거래유형'].replace('-', np.nan)
df['중개사소재지'] = df['중개사소재지'].replace('-', np.nan)
df['본번'] = df['본번'].fillna(-1).astype(int).astype(str)
df['부번'] = df['부번'].fillna(-1).astype(int).astype(str)
df['본번'] = df['본번'].replace("-1", np.nan)
df['부번'] = df['부번'].replace("-1", np.nan)
display(df['본번'])

0          658
1          658
2          658
3          658
4          658
          ... 
1128089    816
1128090    816
1128091    816
1128092    816
1128093    816
Name: 본번, Length: 1128094, dtype: object

In [30]:
df = df.drop(columns=['index'])
display(df)

,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,아파트군,한강변,구,동,계약연도,계약월,구_평균가격순위,구_상승률순위,신축여부,건물연식
0,서울특별시 강남구 개포동,658-1,658,1,개포6차우성,79.97,201712,8,3,1987,...,2,1,강남구,개포동,2017,12,1.0,6.0,0,30
1,서울특별시 강남구 개포동,658-1,658,1,개포6차우성,79.97,201712,22,4,1987,...,2,1,강남구,개포동,2017,12,1.0,6.0,0,30
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성,54.98,201712,28,5,1987,...,2,1,강남구,개포동,2017,12,1.0,6.0,0,30
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성,79.97,201801,3,4,1987,...,2,1,강남구,개포동,2018,1,1.0,6.0,0,31
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성,79.97,201801,8,2,1987,...,2,1,강남구,개포동,2018,1,1.0,6.0,0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,816,0,신내우디안1단지,84.65,202307,19,13,2014,...,1,0,중랑구,신내동,2023,7,22.0,18.0,1,9
1128090,서울특별시 중랑구 신내동,816,816,0,신내우디안1단지,84.62,202307,25,12,2014,...,1,0,중랑구,신내동,2023,7,22.0,18.0,1,9
1128091,서울특별시 중랑구 신내동,816,816,0,신내우디안1단지,101.65,202308,27,12,2014,...,1,0,중랑구,신내동,2023,8,22.0,18.0,1,9
1128092,서울특별시 중랑구 신내동,816,816,0,신내우디안1단지,84.94,202309,2,18,2014,...,1,0,중랑구,신내동,2023,9,22.0,18.0,1,9


In [31]:
df.isnull().sum()

시군구           0
번지          227
본번           75
부번            0
아파트명          0
           ... 
계약월           0
구_평균가격순위      0
구_상승률순위       0
신축여부          0
건물연식          0
Length: 66, dtype: int64

In [32]:
#결측치 처리한 데이터를 훈련 데이터와 테스트 데이터로 분리 후 저장

# train_restore = df[df['is_test'] == 0].reset_index(drop=True)
# test_restore = df[df['is_test'] == 1].reset_index(drop=True)
# train_restore.drop(columns=['is_test'], inplace=True)
# test_restore.drop(columns=['is_test'], inplace=True)
# display(len(house_train))
# display(len(house_test))
# display(len(train_restore))
# display(len(test_restore))
# train_restore.to_csv('./../../../Datas/train_restore.csv', index=False)
# test_restore.to_csv('./../../../Datas/test_restore.csv', index=False)

df.to_csv('./../../../Datas/updated_df2(knn).csv')